### Future-Features
- [ ] Make a provision for self completion status and parent completion status<br>
   <font size = "2">If parent is completed - whole subtask is completed, but if user sets the parent task uncomplete afterwards, set the completion status to what was self completion status<br>
   Use the OR function between the self and parent completion status to achieve this. </font>

- [ ] Add `Repeats` class provisionality

- [ ] Implement `Deadlines` class properlyz

In [4]:
# Imports

import time, math
import numpy as np
from enum import Enum, auto

In [5]:
# Functions for ProgressMetric objects

get_weights = np.vectorize(lambda task: task.weight)

def average(tasks):
    return np.mean(get_weights(tasks))

def softmax(tasks):
    exponentiated_weights = np.exp(get_weights(tasks))
    return exponentiated_weights / np.sum(exponentiated_weights)

def logarithmic_mean(tasks):
    logarithmic_weights = np.log(get_weights(tasks))
    return logarithmic_weights / np.sum(logarithmic_weights)

In [6]:
# Global Variables

min_weight = 0
max_weight = 5

file_logs = "Logs.txt"

class Deadlines(Enum):
    no_deadline = auto()
    date_deadline = auto()
    duration_deadline = auto()

# Removed time_deadline
# Shifted repeat_deadline to be its own thing independent of Deadlines - thus utilizing Repeats class

class Repeats(Enum):
    endless = auto()
    till_date = auto()
    repeat_count = auto()

class ProgressMetric(Enum):
    linear = average
    exponential = softmax
    logarithmic = logarithmic_mean
    

In [7]:
class Common_Functions:
    def log(self, message: str, display: bool = True) -> None:
        timestamp = time.strftime("%I:%M:%S %p, %d %b %Y")
        log_message = f"({timestamp}) {self.name}: {message}\n"

        with open(file_logs, "a") as file:
            file.write(log_message)

        if display:
            print(log_message)

    def log_error(self, error_type, error_message: str) -> None:
        self.log(error_message, False)
        raise error_type(error_message)

In [8]:
class Task(Common_Functions):
    '''
    Instantiate a new `Task` by provding its to-do title.
    ### Required Parameters
    - `name`: Set the work-title you have to do

    ### Additional Parameters
    - `details` (`str`) - Add a description of the task you are doing<br>
    - `weight` (`float`) - Add a *relative* priority value to the task from a range of values<br>
    - `deadline_type` (`Deadlines`) - Add a deadline to your task<br>
    &ensp;&ensp;&ensp;- `no _dealine` - no deadline for the task<br>
    &ensp;&ensp;&ensp;- `date_deadline` - set a date for when the task is due<br>
    &nbsp;&ensp;&ensp;- `duration_deadline` - set a duration in which the task is to be performed<br>
    - `completed` (`bool`) - Tells whether the task is completed or not<br>
    - `parent` (`Task`) - Sets the parent-task of the current task (doesn't implement this task becoming a subtask)<br>
    - `subtasks` (`list(Task)`) - Stores all the subtasks of this task (doesn't implement their parent being current task)

    ### Class Parameters
    - `task_number` (`int`) - Used to make the ID of the current task
    - `tasks_generated` (`int`) - Stores the number of tasks ever generated
    - `tasks_done` (`int`) - Stores the number of tasks ever done
    '''

    task_number = 1
    tasks_generated = 1
    tasks_done = 0

    def __init__(self, name) -> None:
        self.name = name
        self.ID = self.task_number
        self.details = ""
        self.weight = 1
        self.deadline_type = Deadlines.no_deadline

        self.completed = False
        self.progress = 0
        self.impact_calculator = ProgressMetric.linear

        self.parent = None
        self.subtasks = np.array([])

        Task.task_number += 1
        Task.tasks_generated += 1

        self.log(f"Instantiated the task")

    def set_description(self, details: str) -> None:
        self.details = details
        self.log(f"Modified description")

    def set_priority(self, weight: float) -> None:
        if self.weight < min_weight:
            self.log_error(ValueError, f"Weight can't be less than {min_weight}!")

        if self.weight > max_weight:
            self.log_error(ValueError, f"Weight can't be greater than {max_weight}!")

        self.weight = weight
        self.log("Modified Weight")

    def set_deadline(self, deadline_type, deadline_info) -> None:
        match deadline_type:
            case Deadlines.date_deadline.value:
                self.deadline_type = Deadlines.date_deadline
            case Deadlines.time_deadline.value:
                self.deadline_type = Deadlines.time_deadline
            case Deadlines.duration_deadline.value:
                self.deadline_type = Deadlines.duration_deadline
            case Deadlines.repeat_deadline.value:
                self.deadline_type = Deadlines.date_deadline
            case _:
                self.deadline_type = Deadlines.no_deadline

    def has_subtasks(self) -> bool:
        return self.subtasks.size != 0
    
    def has_parent(self) -> bool:
        return True if self.parent is not None else False

    def add_subtask(self, subtask_name: str) -> None:
        new_subtask = Task(subtask_name)
        self.set_subtask(new_subtask)

    def set_subtask(self, subtask) -> None:
        if not isinstance(subtask, Task):
            self.log_error(TypeError, f"Can't make a non-Task object as sub-task!")

        self.subtasks = self.subtasks.append(self.subtasks, subtask)
        self.log(f"Added {subtask.name} as a subtask")

        subtask.parent = self
        subtask.log(f"Set {self.name} as a parent")

    def set_completion_status(self, status: bool, parent_traversal: bool = True, subtask_traversal: bool = True) -> None:
        if not isinstance(status, bool):
            self.log_error(ValueError, f"Can't set the completion status to {status}! (should be True/False)")
        
        previous_status = self.completed
        self.completed = status
        self.log(f"Set completion status to {status}")

        if previous_status ^ status:
            Task.tasks_done += 2 * int(status) - 1

            if self.has_subtasks() and subtask_traversal:
                batch_complete_subtasks = np.vectorize(lambda subtask: subtask.set_completion_status(status, parent_traversal = False))
                batch_complete_subtasks(self.subtasks)
            
            self.update_progress()

            if self.has_parent() and parent_traversal:
                batch_completion_status = np.vectorize(lambda subtask: subtask.completed) 
                subtasks_completed = np.all(batch_completion_status(self.parent.subtasks) == True)
                self.parent.set_completion_status(subtasks_completed, subtask_traversal = False)
    
    def update_progress(self) -> None:
        if self.has_subtasks():
            task_weights = get_weights(self.subtasks)
            subtask_impacts = self.impact_calculator(task_weights)

            get_subtasks_status = np.vectorize(lambda task: task.completed)
            self.progress = np.sum(subtask_impacts * get_subtasks_status(self.subtasks))
        else:
            self.progress = int(self.completed)

        self.log(f"Updated progress to {self.progress}")


#### Testing classes and functioning of production-code

In [31]:
task1 = Task("Complete CS201 Quiz 1")
print(task1.ID)
task1.ID = 3
print(task1.ID)
task1.deadline_type

(05:31:00 PM, 25 Jul 2024) Complete CS201 Quiz 1: Instantiated the task
1
3


<Deadlines.no_deadline: 1>

In [32]:
task2 = Task("Go to Class - HS201")
print(task2.ID)
task1.ID = 10
print(task2.ID)
Task.ID = 1
print(task2.ID)

(05:31:00 PM, 25 Jul 2024) Go to Class - HS201: Instantiated the task
2
2
2


#### Testing Modules and Functions for implementation

In [33]:
class Point:
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y

    def __repr__(self) -> str:
       return f"{self.__class__.__name__}({self.x}, {self.y})"
    
    def scale(self, factor: int | float):
        # print(not isinstance(factor, int), not isinstance(factor, float))
        # if (not isinstance(factor, int)) or (not isinstance(factor, float)):
        #     print("Can't scale a point by non-number quantity!")
        #     return None

        return Point(self.x * factor, self.y * factor)

In [34]:
p1 = Point(2, 3)
p1.scale("1")

Point(11, 111)

In [16]:
print(True ^ True) # False
print(True ^ False) # True
print(False ^ True) # True
print(False ^ False) # False

print(1 ^ 1) # 0
print(1 ^ 2) # 1
print(0 ^ 1) # 1
print(0 ^ 0) # 0

False
True
True
False
0
3
1
0


In [1]:
print(int(True))
print(int(False))

1
0


In [13]:
arr = np.array([True, False, True, True, False])
arr2 = (np.arange(len(arr)) + 1) / 15
print(arr2)
print(arr * arr2)

print(sum(arr2))
print(sum(arr * arr2))

[0.06666667 0.13333333 0.2        0.26666667 0.33333333]
[0.06666667 0.         0.2        0.26666667 0.        ]
1.0
0.5333333333333333


In [19]:
a = np.linspace(0, 1, 100)
print(a)

for e in a:
    print(bool(math.floor(e)), end = "")

[0.         0.01010101 0.02020202 0.03030303 0.04040404 0.05050505
 0.06060606 0.07070707 0.08080808 0.09090909 0.1010101  0.11111111
 0.12121212 0.13131313 0.14141414 0.15151515 0.16161616 0.17171717
 0.18181818 0.19191919 0.2020202  0.21212121 0.22222222 0.23232323
 0.24242424 0.25252525 0.26262626 0.27272727 0.28282828 0.29292929
 0.3030303  0.31313131 0.32323232 0.33333333 0.34343434 0.35353535
 0.36363636 0.37373737 0.38383838 0.39393939 0.4040404  0.41414141
 0.42424242 0.43434343 0.44444444 0.45454545 0.46464646 0.47474747
 0.48484848 0.49494949 0.50505051 0.51515152 0.52525253 0.53535354
 0.54545455 0.55555556 0.56565657 0.57575758 0.58585859 0.5959596
 0.60606061 0.61616162 0.62626263 0.63636364 0.64646465 0.65656566
 0.66666667 0.67676768 0.68686869 0.6969697  0.70707071 0.71717172
 0.72727273 0.73737374 0.74747475 0.75757576 0.76767677 0.77777778
 0.78787879 0.7979798  0.80808081 0.81818182 0.82828283 0.83838384
 0.84848485 0.85858586 0.86868687 0.87878788 0.88888889 0.89898

In [20]:
print(np.all([True, True]))
print(np.all([True, False]))
print(np.all([False, True]))
print(np.all([False, False]))

True
False
False
False
